<a href="https://colab.research.google.com/github/Eepshaa/From-Speed-to-Sustainability/blob/main/Actual_Model_with_Random_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Random Data**

In [ ]:
import pandas as pd
import numpy as np

# Set random seed for reproducibility
np.random.seed(42)

# Parameters
num_couriers = 500
num_orders = 10000
hours = list(range(24))  # 0 to 23 hours of the day

# Time of day classifications based on the hour
def classify_time_of_day(hour):
    if 7 <= hour < 9:
        return 'Morning Rush Hour'
    elif 10 <= hour < 15:
        return 'Midday'
    elif 16 <= hour < 19:
        return 'Evening Rush Hour'
    else:
        return 'Night'

# Generate data for Starting Points Table
courier_ids = [f'C{i:03d}' for i in range(1, num_couriers + 1)]
expanded_data = []

for hour in hours:
    time_of_day = classify_time_of_day(hour)
    for courier_id in courier_ids:
        location_x = np.random.randint(0, 100)
        location_y = np.random.randint(0, 100)
        availability = np.random.randint(0, 2)  # 0 or 1 availability
        expanded_data.append([courier_id, location_x, location_y, hour, time_of_day, availability])

expanded_starting_points_df = pd.DataFrame(expanded_data, columns=[
    'Courier ID', 'Location x', 'Location y', 'Hour', 'Time of Day', 'Courier Availability'
])

# Generate data for Courier Type Table
vehicle_types = ['Bicycle', 'Car', 'Motorcycle']
courier_type_df = pd.DataFrame({
    'Courier ID': courier_ids,
    'Courier Vehicle Type': np.random.choice(vehicle_types, size=num_couriers),
    'Courier Vehicle': ['Vehicle' + str(i) for i in range(1, num_couriers + 1)],
    'Environmental Impact': np.nan  # Initialize with NaN
})

courier_type_df['Environmental Impact'] = np.where(
    courier_type_df['Courier Vehicle Type'] == 'Bicycle',
    0.0,
    np.random.uniform(0.1, 0.5, size=num_couriers)
)

# Generate orders distribution
morning_orders = int(num_orders * 0.35)
noon_orders = int(num_orders * 0.40)
night_orders = int(num_orders * 0.25)

# Generate data for Orders Table
morning_hours = np.random.normal(loc=9, scale=1, size=morning_orders).astype(int) % 24
noon_hours = np.random.normal(loc=13, scale=1, size=noon_orders).astype(int) % 24
night_hours = np.random.normal(loc=19, scale=1, size=night_orders).astype(int) % 24

# Combine all generated hours
order_hours = np.concatenate([morning_hours, noon_hours, night_hours])

# Ensure the total number of orders matches the num_orders
order_hours = order_hours[:num_orders]

orders_df = pd.DataFrame({
    'Order ID': [f'O{i:05d}' for i in range(1, num_orders + 1)],
    'Start Location X': np.random.randint(0, 100, size=num_orders),
    'Start Location Y': np.random.randint(0, 100, size=num_orders),
    'End Location X': np.random.randint(0, 100, size=num_orders),
    'End Location Y': np.random.randint(0, 100, size=num_orders),
    'Hour': order_hours
})

# Classify the time of day for each order
orders_df['Time of Day'] = orders_df['Hour'].apply(classify_time_of_day)

# Display the resulting DataFrames
print(expanded_starting_points_df.head())
print(courier_type_df.head())
print(orders_df.head())


  Courier ID  Location x  Location y  Hour Time of Day  Courier Availability
0       C001          51          92     0       Night                     0
1       C002          71          60     0       Night                     0
2       C003          82          86     0       Night                     0
3       C004          74          87     0       Night                     0
4       C005          99          23     0       Night                     0
  Courier ID Courier Vehicle Type Courier Vehicle  Environmental Impact
0       C001           Motorcycle        Vehicle1              0.314159
1       C002              Bicycle        Vehicle2              0.000000
2       C003                  Car        Vehicle3              0.187002
3       C004           Motorcycle        Vehicle4              0.384794
4       C005                  Car        Vehicle5              0.465707
  Order ID  Start Location X  Start Location Y  End Location X  \
0   O00001                96            

# **Actual Data**

In [ ]:
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')

# Path to the CSV file
file_path = '/content/drive/MyDrive/AI_Implementation_Shared_Drive/Data_Files/all_waybill_info_meituan.csv'

# Read CSV file into a DataFrame
all_data_df = pd.read_csv(file_path)

# List of time-related columns in 'all_data_df' that need conversion
time_columns = [
    'estimate_arrived_time', 'dispatch_time', 'grab_time',
    'fetch_time', 'arrive_time', 'order_push_time', 'platform_order_time','estimate_meal_prepare_time'
]

# Convert each time-related column from Unix timestamp to datetime
for column in time_columns:
    all_data_df[column] = pd.to_datetime(all_data_df[column], unit='s')

# Path to the CSV file
file_path = '/content/drive/MyDrive/AI_Implementation_Shared_Drive/Data_Files/courier_wave_info_meituan.csv'

# Read CSV file into a DataFrame
courier_wave_df = pd.read_csv(file_path)

# Convert the 'dispatch_time' column from Unix timestamp to datetime
courier_wave_df['wave_start_time'] = pd.to_datetime(courier_wave_df['wave_start_time'], unit='s')

courier_wave_df['wave_end_time'] = pd.to_datetime(courier_wave_df['wave_end_time'], unit='s')

# Path to the CSV file
file_path = '/content/drive/MyDrive/AI_Implementation_Shared_Drive/Data_Files/dispatch_rider_meituan.csv'

# Read CSV file into a DataFrame
dispatch_rider_df = pd.read_csv(file_path)

# Convert the 'dispatch_time' column from Unix timestamp to datetime
dispatch_rider_df['dispatch_time'] = pd.to_datetime(dispatch_rider_df['dispatch_time'], unit='s')

# Path to the CSV file
file_path = '/content/drive/MyDrive/AI_Implementation_Shared_Drive/Data_Files/2021_Cars_Aggregated.csv'

# Read CSV file into a DataFrame
cars_aggregated_df = pd.read_csv(file_path)




Mounted at /content/drive


In [ ]:

# Reference date to compare against
reference_date = pd.to_datetime('1970-01-01 00:00:00')

# Drop rows where any time column has the reference date
for column in time_columns:
    all_data_df = all_data_df[all_data_df[column] != reference_date].reset_index(drop=True)

# Display the first few rows to verify
print(all_data_df.columns)

# Ensure 'order_push_time' and 'arrive_time' are in datetime format
all_data_df['order_push_time'] = pd.to_datetime(all_data_df['order_push_time'])
all_data_df['arrive_time'] = pd.to_datetime(all_data_df['arrive_time'])

# Extract the date from 'order_push_time' for grouping
all_data_df['date'] = all_data_df['order_push_time'].dt.date

# Group by 'courier_id' and 'date' to find min 'order_push_time' and max 'arrive_time'
courier_availability = all_data_df.groupby(['courier_id', 'date']).agg(
    min_order_push_time=('order_push_time', 'min'),
    max_arrive_time=('arrive_time', 'max')
).reset_index()

# Separate date and time components
courier_availability['min_order_push_hour'] = courier_availability['min_order_push_time'].dt.time
courier_availability['max_arrive_hour'] = courier_availability['max_arrive_time'].dt.time

# Drop the original datetime columns if only time is needed
courier_availability.drop(columns=['min_order_push_time', 'max_arrive_time'], inplace=True)


# Function to adjust time by adding or subtracting 1 hour and extract the hour
def adjust_and_extract_hour(time, add_hours=0):
    if pd.isna(time):
        return np.nan
    # Create a Timestamp object for easy manipulation
    adjusted_time = pd.Timestamp('1900-01-01') + pd.Timedelta(hours=time.hour + add_hours)
    # Extract and return the hour
    return adjusted_time.hour

# Apply the adjustment:
# Subtract 1 hour from 'min_order_push_hour' and extract the hour
courier_availability['min_order_push_hour'] = courier_availability['min_order_push_hour'].apply(lambda x: adjust_and_extract_hour(x, add_hours=-1))

# Add 1 hour to 'max_arrive_hour' and extract the hour
courier_availability['max_arrive_hour'] = courier_availability['max_arrive_hour'].apply(lambda x: adjust_and_extract_hour(x, add_hours=1))

# Display the updated courier availability table
print(courier_availability)

Index(['Unnamed: 0', 'dt', 'order_id', 'waybill_id', 'courier_id', 'da_id',
       'is_courier_grabbed', 'is_weekend', 'estimate_arrived_time',
       'is_prebook', 'poi_id', 'sender_lng', 'sender_lat', 'recipient_lng',
       'recipient_lat', 'grab_lng', 'grab_lat', 'dispatch_time', 'grab_time',
       'fetch_time', 'estimate_meal_prepare_time', 'arrive_time',
       'order_push_time', 'platform_order_time'],
      dtype='object')
       courier_id        date  min_order_push_hour  max_arrive_hour
0               0  2022-10-17                   12               16
1               0  2022-10-18                   13               16
2               0  2022-10-20                   13               17
3               0  2022-10-23                   13               16
4               1  2022-10-16                   15               17
...           ...         ...                  ...              ...
29696        4945  2022-10-19                    9               12
29697        4946  2

In [ ]:
import pandas as pd

# Assume all_data_df is already defined and loaded

# Convert time columns to datetime
for time_col in ['dispatch_time', 'grab_time', 'fetch_time', 'arrive_time']:
    all_data_df[time_col] = pd.to_datetime(all_data_df[time_col])

# Calculate total time taken for each order
all_data_df['total_time'] = (all_data_df['arrive_time'] - all_data_df['dispatch_time']).dt.total_seconds()

# Aggregate time metrics by courier_id
courier_stats = all_data_df.groupby('courier_id')['total_time'].mean().reset_index()

# Determine the 33rd percentile value
threshold = courier_stats['total_time'].quantile(0.66)

# Identify the slowest 33% of couriers
slowest_couriers = courier_stats[courier_stats['total_time'] > threshold]['courier_id'].tolist()

courier_availability['courier_type'] = courier_availability['courier_id'].apply(lambda x: 'Bicycle' if x in slowest_couriers else 'N/A')
print(courier_availability)

       courier_id        date  min_order_push_hour  max_arrive_hour  \
0               0  2022-10-17                   12               16   
1               0  2022-10-18                   13               16   
2               0  2022-10-20                   13               17   
3               0  2022-10-23                   13               16   
4               1  2022-10-16                   15               17   
...           ...         ...                  ...              ...   
29696        4945  2022-10-19                    9               12   
29697        4946  2022-10-20                    5               11   
29698        4950  2022-10-19                   11               14   
29699        4953  2022-10-21                    6               10   
29700        4954  2022-10-24                    5                8   

      courier_type  
0              N/A  
1              N/A  
2              N/A  
3              N/A  
4              N/A  
...            ...  


In [ ]:
import numpy as np
import pandas as pd

# Create 'car_type' column by merging 'Manufacturer' and 'Fuel Type'
cars_aggregated_df['car_type'] = cars_aggregated_df['Manufacturer'] + ' ' + cars_aggregated_df['Fuel Type']

# Define a function to assign car types based on the distribution
def assign_car_types(courier_df, car_distribution):
    # Filter out couriers with 'N/A' in 'courier_type'
    na_couriers = courier_df[courier_df['courier_type'] == 'N/A']

    if not na_couriers.empty:
        # Create a list of car types and their distribution
        car_types = car_distribution['car_type'].tolist()
        probabilities = car_distribution['Number of vehicles'].tolist()

        # Normalize probabilities
        probabilities = np.array(probabilities) / sum(probabilities)

        # Randomly assign car types to each 'N/A' courier based on the distribution
        courier_df.loc[courier_df['courier_type'] == 'N/A', 'car_type'] = np.random.choice(car_types, size=len(na_couriers), p=probabilities)

# Apply the function to assign car types to 'N/A' couriers
assign_car_types(courier_availability, cars_aggregated_df)

# Fill 'car_type' with 'Bicycle' where it is missing
courier_availability['car_type'].fillna('Bicycle', inplace=True)

# Determine the final vehicle type for each courier_id
# Prioritize 'Bicycle' if it is the only option or if the 'car_type' is still missing
courier_availability['courier_vehicle_type'] = courier_availability.groupby('courier_id')['car_type'].transform(lambda x: x.mode()[0])

# Drop the original columns if no longer needed
courier_availability.drop(columns=['courier_type', 'car_type'], inplace=True)

# Display the updated DataFrame
print(courier_availability)



       courier_id        date  min_order_push_hour  max_arrive_hour  \
0               0  2022-10-17                   12               16   
1               0  2022-10-18                   13               16   
2               0  2022-10-20                   13               17   
3               0  2022-10-23                   13               16   
4               1  2022-10-16                   15               17   
...           ...         ...                  ...              ...   
29696        4945  2022-10-19                    9               12   
29697        4946  2022-10-20                    5               11   
29698        4950  2022-10-19                   11               14   
29699        4953  2022-10-21                    6               10   
29700        4954  2022-10-24                    5                8   

                   courier_vehicle_type  
0      MERCEDES-BENZ AG DIESEL/ELECTRIC  
1      MERCEDES-BENZ AG DIESEL/ELECTRIC  
2      MERCEDES-BENZ 

In [ ]:
import numpy as np

# Find min and max latitude and longitude
min_lat = all_data_df[['sender_lat', 'recipient_lat', 'grab_lat']].min().min()
max_lat = all_data_df[['sender_lat', 'recipient_lat', 'grab_lat']].max().max()

min_lng = all_data_df[['sender_lng', 'recipient_lng', 'grab_lng']].min().min()
max_lng = all_data_df[['sender_lng', 'recipient_lng', 'grab_lng']].max().max()

# Display the results
print(f"Minimum Latitude: {min_lat}")
print(f"Maximum Latitude: {max_lat}")
print(f"Minimum Longitude: {min_lng}")
print(f"Maximum Longitude: {max_lng}")



# Function to generate random latitude and longitude
def generate_random_position(min_lat, max_lat, min_lng, max_lng):
    random_lat = np.random.uniform(min_lat, max_lat)
    random_lng = np.random.uniform(min_lng, max_lng)
    return random_lat, random_lng

# Apply the function to add random positions to courier_availability DataFrame
courier_availability[['courier_lat', 'courier_lng']] = courier_availability.apply(
    lambda row: pd.Series(generate_random_position(min_lat, max_lat, min_lng, max_lng)),
    axis=1
)

# Display the updated courier availability table with random positions
print(courier_availability)



Minimum Latitude: 45729124
Maximum Latitude: 46079853
Minimum Longitude: 174382615
Maximum Longitude: 174999535
       courier_id        date  min_order_push_hour  max_arrive_hour  \
0               0  2022-10-17                   12               16   
1               0  2022-10-18                   13               16   
2               0  2022-10-20                   13               17   
3               0  2022-10-23                   13               16   
4               1  2022-10-16                   15               17   
...           ...         ...                  ...              ...   
29696        4945  2022-10-19                    9               12   
29697        4946  2022-10-20                    5               11   
29698        4950  2022-10-19                   11               14   
29699        4953  2022-10-21                    6               10   
29700        4954  2022-10-24                    5                8   

                   courier_vehicle_

In [ ]:
# Create a mapping from car_type to WLTP CO2 emissions (g/km)
co2_emissions_mapping = cars_aggregated_df[['car_type', 'WLTP CO2 emissions (g/km)']].drop_duplicates()
co2_emissions_mapping.set_index('car_type', inplace=True)


# Map CO2 emissions to the courier_availability DataFrame
courier_availability['WLTP_CO2_emissions'] = courier_availability['courier_vehicle_type'].map(co2_emissions_mapping['WLTP CO2 emissions (g/km)'])

# Set CO2 emissions to 0 for bicycles
courier_availability.loc[courier_availability['courier_vehicle_type'] == 'Bicycle', 'WLTP_CO2_emissions'] = 0

# Display the updated DataFrame
print(courier_availability)

       courier_id        date  min_order_push_hour  max_arrive_hour  \
0               0  2022-10-17                   12               16   
1               0  2022-10-18                   13               16   
2               0  2022-10-20                   13               17   
3               0  2022-10-23                   13               16   
4               1  2022-10-16                   15               17   
...           ...         ...                  ...              ...   
29696        4945  2022-10-19                    9               12   
29697        4946  2022-10-20                    5               11   
29698        4950  2022-10-19                   11               14   
29699        4953  2022-10-21                    6               10   
29700        4954  2022-10-24                    5                8   

                   courier_vehicle_type   courier_lat   courier_lng  \
0      MERCEDES-BENZ AG DIESEL/ELECTRIC  4.586030e+07  1.744132e+08   
1    

In [ ]:
import pandas as pd

# Define the columns relevant to order positions and platform order time
order_position_columns = ['order_id', 'sender_lat', 'sender_lng', 'recipient_lat', 'recipient_lng', 'platform_order_time']

# Extract these columns from all_data_df
order_positions_df = all_data_df[order_position_columns]

# Rename columns for clarity
order_positions_df.rename(columns={
    'sender_lat': 'order_start_lat',
    'sender_lng': 'order_start_lng',
    'recipient_lat': 'order_end_lat',
    'recipient_lng': 'order_end_lng'
}, inplace=True)

# Convert platform_order_time to datetime format
order_positions_df['platform_order_time'] = pd.to_datetime(order_positions_df['platform_order_time'])

# Extract day, hour, and minute from platform_order_time
order_positions_df['platform_order_day'] = order_positions_df['platform_order_time'].dt.date
order_positions_df['platform_order_hour'] = order_positions_df['platform_order_time'].dt.hour
order_positions_df['platform_order_minute'] = order_positions_df['platform_order_time'].dt.minute

# Format hour and minute as HH:MM
order_positions_df['platform_order_time_formatted'] = order_positions_df['platform_order_hour'].astype(str).str.zfill(2) + ':' + order_positions_df['platform_order_minute'].astype(str).str.zfill(2)

# Drop the original platform_order_time if not needed
order_positions_df.drop(columns=['platform_order_time','platform_order_hour','platform_order_minute'], inplace=True)

# Display the first few rows of the updated DataFrame
print(order_positions_df)



<ipython-input-7-f665b5f01a36>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  order_positions_df.rename(columns={
<ipython-input-7-f665b5f01a36>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  order_positions_df['platform_order_time'] = pd.to_datetime(order_positions_df['platform_order_time'])
<ipython-input-7-f665b5f01a36>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/us

        order_id  order_start_lat  order_start_lng  order_end_lat  \
0              1         45905850        174529930       45898250   
1              2         45861460        174532407       45891243   
2              3         45868015        174559718       45886787   
3              4         45868015        174559718       45867411   
4              5         45920329        174542688       45921310   
...          ...              ...              ...            ...   
568539    568542         45879427        174650494       45876472   
568540    568543         45868634        174575022       45862730   
568541    568544         45879398        174650478       45873834   
568542    162081         45889549        174535543       45887366   
568543    568545         45887997        174533308       45899319   

        order_end_lng platform_order_day platform_order_time_formatted  
0           174522983         2022-10-16                         15:59  
1           174547033    

<ipython-input-7-f665b5f01a36>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  order_positions_df['platform_order_time_formatted'] = order_positions_df['platform_order_hour'].astype(str).str.zfill(2) + ':' + order_positions_df['platform_order_minute'].astype(str).str.zfill(2)
<ipython-input-7-f665b5f01a36>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  order_positions_df.drop(columns=['platform_order_time','platform_order_hour','platform_order_minute'], inplace=True)


In [ ]:
import pandas as pd
from sklearn.cluster import DBSCAN
import numpy as np

# Step 1: Sort the DataFrame by latitude and longitude
sorted_df = order_positions_df.sort_values(by=['order_start_lat', 'order_start_lng'])

# Step 2: Split the sorted data into batches of 100 and take the average
batch_size = 100
batches = [sorted_df[i:i + batch_size] for i in range(0, len(sorted_df), batch_size)]

# Create a list to store the averages
averaged_list = []

for batch in batches:
    avg_lat = batch['order_start_lat'].mean()
    avg_lng = batch['order_start_lng'].mean()
    averaged_list.append({'lat': avg_lat, 'lng': avg_lng})

# Convert the list of averages to a DataFrame
averaged_df = pd.DataFrame(averaged_list)

# Step 3: Apply DBSCAN to the averaged data
db = DBSCAN(eps=3000, min_samples=100, metric='euclidean').fit(averaged_df[['lat', 'lng']])
averaged_df['cluster'] = db.labels_

# Step 4: Calculate the number of points in each cluster
cluster_sizes = averaged_df['cluster'].value_counts().reset_index()
cluster_sizes.columns = ['cluster', 'size']

# Merge cluster sizes with city center coordinates
city_centers = averaged_df[averaged_df['cluster'] != -1].groupby('cluster').apply(lambda x: x[['lat', 'lng']].mean()).reset_index()
city_centers.columns = ['cluster_id', 'lat', 'lng']  # Renaming 'cluster' to 'cluster_id' to avoid conflicts

# Merge with cluster sizes
city_centers = city_centers.merge(cluster_sizes, left_on='cluster_id', right_on='cluster', how='left').drop(columns=['cluster'])

# Function to classify zones
def classify_zone(row, core_points, eps):
    distances = np.sqrt((core_points['lat'] - row['lat'])**2 + (core_points['lng'] - row['lng'])**2)
    return 'Nearby' if any(distances <= eps) else 'Far'

# Add a new column to classify zones
averaged_df['zone'] = averaged_df.apply(classify_zone, axis=1, core_points=averaged_df[averaged_df['cluster'] != -1], eps=0.5)

# Step 5: Save city zones in a different DataFrame
city_zones_df = averaged_df[['lat', 'lng', 'zone', 'cluster']].copy()

# Print results
print("City Centers with Cluster Sizes:")
print(city_centers)

print("City Zones DataFrame:")
print(city_zones_df.head())







City Centers with Cluster Sizes:
   cluster_id           lat           lng  size
0           0  4.586086e+07  1.745771e+08   601
1           1  4.586567e+07  1.745580e+08   107
2           2  4.586893e+07  1.745729e+08   103
3           3  4.587692e+07  1.745461e+08   167
4           4  4.589629e+07  1.745544e+08   390
5           5  4.590606e+07  1.745292e+08   138
6           6  4.592033e+07  1.745426e+08   130
7           7  4.605206e+07  1.749445e+08   188
City Zones DataFrame:
           lat           lng zone  cluster
0  45757017.07  1.745698e+08  Far       -1
1  45761181.57  1.745669e+08  Far       -1
2  45762760.00  1.745663e+08  Far       -1
3  45763002.80  1.745663e+08  Far       -1
4  45763075.52  1.745664e+08  Far       -1


In [ ]:
print(city_centers_df)

NameError: name 'city_centers_df' is not defined

In [ ]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Assuming the dataframes are already loaded:
# expanded_starting_points_df
# courier_type_df
# orders_df

# Function to calculate Euclidean distance
def calculate_distance(x1, y1, x2, y2):
    return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)

# Function to calculate the travel time for a courier
def calculate_travel_time(distance, speed):
    return distance / speed

# Traffic delay function (example)
def traffic_delay(distance, base_time):
    return 0.5 * distance * base_time  # Simplified traffic delay model

# Function to calculate environmental impact
def calculate_environmental_impact(time, distance, vehicle_type, e_time_c, e_distance_c):
    if vehicle_type == 'Bicycle':
        return 0.0
    else:
        E_traffic = time * e_time_c
        E_distance = distance * e_distance_c
        return E_traffic + E_distance

# Objective functions for optimization
def objective_bicycle(order_row, courier_row):
    distance = calculate_distance(order_row['Start Location X'], order_row['Start Location Y'],
                                  order_row['End Location X'], order_row['End Location Y'])
    travel_time = calculate_travel_time(distance, 15)  # Assuming speed = 15 for bicycles
    return travel_time

def objective_car_motorcycle(order_row, courier_row):
    distance = calculate_distance(order_row['Start Location X'], order_row['Start Location Y'],
                                  order_row['End Location X'], order_row['End Location Y'])
    base_time = calculate_travel_time(distance, 40)  # Assuming speed = 40 for cars/motorcycles
    total_time = base_time + traffic_delay(distance, base_time)
    return total_time

def objective_environmental(order_row, courier_row):
    distance = calculate_distance(order_row['Start Location X'], order_row['Start Location Y'],
                                  order_row['End Location X'], order_row['End Location Y'])
    base_time = calculate_travel_time(distance, 40)  # Assuming speed = 40 for cars/motorcycles
    total_time = base_time + traffic_delay(distance, base_time)
    e_time_c = courier_row['Environmental Impact']
    e_distance_c = courier_row['Environmental Impact']
    return calculate_environmental_impact(total_time, distance, courier_row['Courier Vehicle Type'],
                                          e_time_c, e_distance_c)

# Function to find the optimal courier for an order
def find_optimal_courier(order_row, couriers_df, objective_func):
    best_courier = None
    best_value = float('inf')

    for _, courier_row in couriers_df.iterrows():
        value = objective_func(order_row, courier_row)
        if value < best_value:
            best_value = value
            best_courier = courier_row

    return best_courier, best_value

# Iterate through orders and find the optimal courier for each
detailed_assignments = []
for _, order_row in orders_df.iterrows():
    time_interval = order_row['Time']

    # Filter couriers available at the order's time interval
    available_couriers = expanded_starting_points_df[
        (expanded_starting_points_df['Time Interval'] == time_interval) &
        (expanded_starting_points_df['Courier Availability'] == 1)
    ]

    # Merge with courier type information
    available_couriers = available_couriers.merge(courier_type_df, on='Courier ID')

    # Find the optimal courier by different objectives
    optimal_bicycle, bicycle_time = find_optimal_courier(order_row, available_couriers[available_couriers['Courier Vehicle Type'] == 'Bicycle'], objective_bicycle)
    optimal_car_motorcycle, car_motorcycle_time = find_optimal_courier(order_row, available_couriers[available_couriers['Courier Vehicle Type'] != 'Bicycle'], objective_car_motorcycle)
    optimal_environmental, environmental_impact = find_optimal_courier(order_row, available_couriers[available_couriers['Courier Vehicle Type'] != 'Bicycle'], objective_environmental)

    # Determine if the bicycle is the fastest option
    if optimal_bicycle is not None and (
        (optimal_car_motorcycle is None or bicycle_time < car_motorcycle_time) and
        (optimal_environmental is None or bicycle_time < environmental_impact)
    ):
        # If the bicycle is the fastest, choose it as the optimal for all objectives
        best_courier = optimal_bicycle
        best_time = bicycle_time
        best_environmental_impact = 0.0  # Bicycles have no environmental impact
        best_courier_for_time = best_courier_for_environmental = best_courier
    else:
        # Check if the least time vehicle is the same as the least environmental impact vehicle
        if optimal_car_motorcycle is not None and optimal_environmental is not None and optimal_car_motorcycle['Courier ID'] == optimal_environmental['Courier ID']:
            # If they are the same, set both to the same courier
            best_courier_for_time = best_courier_for_environmental = optimal_car_motorcycle
            best_time = car_motorcycle_time
            best_environmental_impact = environmental_impact
        else:
            # Use original logic
            best_courier_for_time = optimal_car_motorcycle
            best_time = car_motorcycle_time
            best_courier_for_environmental = optimal_environmental
            best_environmental_impact = environmental_impact

    detailed_assignments.append({
        'Order ID': order_row['Order ID'],
        'Order Time': order_row['Time'],
        'Optimal Courier for Time': best_courier_for_time['Courier ID'] if best_courier_for_time is not None else None,
        'Arrival Time': best_time if best_courier_for_time is not None else None,
        'Optimal Courier for Environmental Impact': best_courier_for_environmental['Courier ID'] if best_courier_for_environmental is not None else None,
        'Environmental Impact': best_environmental_impact if best_courier_for_environmental is not None else None,
        'Courier Vehicle Type': best_courier_for_time['Courier Vehicle Type'] if best_courier_for_time is not None else None,
        'Courier Vehicle': best_courier_for_time['Courier Vehicle'] if best_courier_for_time is not None else None
    })

# Convert the results to a DataFrame for analysis
detailed_assignments_df = pd.DataFrame(detailed_assignments)

# Save the final DataFrame as a CSV file
detailed_assignments_df.to_csv('detailed_assignments_with_impact_and_times.csv', index=False)

# Display the resulting DataFrame
detailed_assignments_df.head()





,Order ID,Order Time,Optimal Courier for Time,Arrival Time,Optimal Courier for Environmental Impact,Environmental Impact,Courier Vehicle Type,Courier Vehicle
0,O00001,Afternoon,C001,4.861184,C001,0.0,Bicycle,Vehicle1
1,O00002,Night,C001,3.887301,C001,0.0,Bicycle,Vehicle1
2,O00003,Evening,C014,3.783590,C014,0.0,Bicycle,Vehicle14
3,O00004,Morning,C008,6.767570,C008,0.0,Bicycle,Vehicle8
4,O00005,Night,C001,5.401646,C001,0.0,Bicycle,Vehicle1
